# Librería SQLite - Interfase para SQLite BD

Repositorio Github.-  **Guia_Python**

Carpeta.- **Módulos/Libreria sqlite.ipynb**

_____________________________________________

Funciones:

- Importar librería **sqlite3**
- Conexión Bd -> **bd = sqlite3.connect(fichero)**
- Crear objeto cursor -> **cursor = bd.cursor()**
- Crear tabla con sequence autoincremental -> **bd.execute("CREATE TABLE articulos (codigo integer primary key autoincrement, ...)")**
- Insertar fila en tabla -> **bd.execute("INSERT INTO articulos(c1, c2) VALUES (?,?)", (v1, v2))**
- Insertar lista de filas en tabla -> **bd.executemany("INSERT INTO articulos (c1, c2) VALUES (?,?)", lista_tuplas)
- Commit, actualizar cambios en fichero -> **bd.commit()**
- Consultar registros 1 salida -> **bd.execute("SELECT c1, c2 FROM articulos WHERE codigo=?", (v1, )).fetchone()**
- Consulta varios registros (iterable) -> **bd.execute('SELECT * FROM unidades;')**
- Consulta varios registros (iterable) 2 -> **bd.execute('SELECT * FROM unidades;').fetchall()**
- IMPORTANTE - Cerrar BD al finalizar -> **bd.close()**

- Recuperar las tablas existentes en la BD SQLite -> **bd.execute('SELECT * FROM sqlite_master where type=?', ('table', ))**

DATAFRAME

  - Leer registros de la tabla y cargar en dataframe -> **pd.read_sql_query('SELECT * FROM unidades', bd)**
  - Grabar dataframe en tabla -> **dataframe.to_sql("tabla", bd, if_exists="replace")**

In [18]:
import sqlite3

### Funcionamiento básico -> Conexión BD y crear objeto cursor

In [33]:
# Crea un objeto de conexión a la base de datos SQLite
fichero = "D:\Python\Espacio\Fuentes\Espacio 1.0 Tablas\espacio.db"
bd = sqlite3.connect(fichero, detect_types=0)       # detecta los tipos python por defecto

# Con la conexión, crea un objeto cursor
cursor = bd.cursor()

### Crear tabla con clave autoincremental

In [24]:
try:
    bd.execute("""create table articulos (
                              codigo integer primary key autoincrement,
                              descripcion text,
                              precio real
                        )""")
    print("se creo la tabla articulos")                        
except sqlite3.OperationalError:
    print("La tabla articulos ya existe")

La tabla articulos ya existe


### Insertar filas en una tabla

In [30]:
bd.execute("insert into articulos(descripcion,precio) values (?,?)", ("naranjas", 23.50))
bd.execute("insert into articulos(descripcion,precio) values (?,?)", ("peras", 34))
bd.execute("insert into articulos(descripcion,precio) values (?,?)", ("bananas", 25))
bd.commit()

### Insertar varias filas de una lista en una tabla

In [32]:
datos = [("mandarinas", 7.9),
        ("manzanas", 6.5),
        ("platanos", 8.0)      ]

bd.executemany("INSERT INTO articulos (descripcion, precio) VALUES (?,?)", datos)
bd.commit()

### Ejecutar sentencias SQL (consultas)

- Leer registros recuperados.- 1) iterando cursor o 2) usando cursor.fetchall()
- Leer un único registro -> cursor.fetchone()

In [27]:
#  Lectura de un único registro con código 2
codigo = 2
cursor = bd.execute("select descripcion,precio from articulos where codigo=?", (codigo, ))

fila = cursor.fetchone()

if fila!=None:
    print(fila)
else:
    print("No existe un artículo con dicho código.")

('peras', 34.0)


In [38]:
# El resultado de "cursor.execute" puede ser iterado por fila
for fila in bd.execute('SELECT * FROM unidades;'):
    print(fila)

# Otra forma de leer el resultado
print("> cursor.fetchall")
for fila in bd.execute('SELECT * FROM unidades;').fetchall():
    print(fila)

('4892-21E4', 'CLOUD CLARA', '2017-08-08 13:20:24', 3002, 221, 5811170010, 804375666688, '2017-08-26 20:07:14')
('AFA9-B42E', 'Cloud Multimedia', '2017-08-26 20:20:40', 37, 4, 42817224474, 804107075584, '2017-08-26 20:33:53')
('D023-D4A5', 'CLOUD JAVI', '2017-08-21 11:37:47', 100, 17, 209713002, 804105539584, '2017-08-26 20:35:33')
('8889-6A42', 'Copia seguridad', '2015-10-16 18:55:12', 2903, 185, 66889898331, 804097355776, '2017-08-26 20:45:37')
('E308-5792', 'Descargas', '', 0, 0, 0, 804077486080, '2017-08-26 21:10:28')
('37AF-A9CA', 'Multimedia', '2016-03-08 20:04:13', 284, 25, 285190378274, 804076920832, '2017-08-26 21:10:58')
('EDDC-A872', 'JAVI', '', 80947, 10693, 327462662318, 0, '2017-08-26 21:19:38')
> cursor.fetchall
('4892-21E4', 'CLOUD CLARA', '2017-08-08 13:20:24', 3002, 221, 5811170010, 804375666688, '2017-08-26 20:07:14')
('AFA9-B42E', 'Cloud Multimedia', '2017-08-26 20:20:40', 37, 4, 42817224474, 804107075584, '2017-08-26 20:33:53')
('D023-D4A5', 'CLOUD JAVI', '2017-08-

### Lectura de registros a dataframe

In [15]:
import pandas as pd

# Usa read_sql_query de pandas para extraer la consulta a DataFrame
df = pd.read_sql_query('SELECT * FROM unidades', bd)

print(df.head())

    numserie           volumen             fechamod  numfiles  numdir  \
0  4892-21E4       CLOUD CLARA  2017-08-08 13:20:24      3002     221   
1  AFA9-B42E  Cloud Multimedia  2017-08-26 20:20:40        37       4   
2  D023-D4A5        CLOUD JAVI  2017-08-21 11:37:47       100      17   
3  8889-6A42   Copia seguridad  2015-10-16 18:55:12      2903     185   
4  E308-5792         Descargas                              0       0   

        tamano         libre            fechaudit  
0   5811170010  804375666688  2017-08-26 20:07:14  
1  42817224474  804107075584  2017-08-26 20:33:53  
2    209713002  804105539584  2017-08-26 20:35:33  
3  66889898331  804097355776  2017-08-26 20:45:37  
4            0  804077486080  2017-08-26 21:10:28  


### Grabar registros de Dataframe en SQLite (nueva tabla)

In [17]:
# Selecciona sólo datos que volumen contenga CLOUD
volumenCLOUD = df[df.volumen.str.contains('CLOUD')]
# planets[planets.method.str.contains('pulsa')]

# Escribe los datos del nuevo DataFrame en una nueva tabla en SQLite
volumenCLOUD.to_sql("unidadesCloud", bd, if_exists="replace")

### Consultar tablas de la BD SQLite

In [39]:
# SELECT * FROM sqlite_master WHERE type = "table";
#  Consultar las tablas existentes en la BD
for tabla in bd.execute('SELECT * FROM sqlite_master where type=?', ('table', )):
    print(tabla)

('table', 'archivos', 'archivos', 2, 'CREATE TABLE archivos\n            (numserie int, path text, nombre text, extension text, tipo text, fechamod text,tamano int, hash int, fechaudit text)')
('table', 'directorios', 'directorios', 3, 'CREATE TABLE directorios\n            (numserie int, path text, fechamod text, numfiles int, numdir int, tamano int, fechaudit text)')
('table', 'unidades', 'unidades', 4, 'CREATE TABLE unidades\n            (numserie text, volumen text, fechamod text, numfiles int, numdir int, tamano int, libre int, fechaudit text)')
('table', 'unidadesCloud', 'unidadesCloud', 1970, 'CREATE TABLE "unidadesCloud" (\n"index" INTEGER,\n  "numserie" TEXT,\n  "volumen" TEXT,\n  "fechamod" TEXT,\n  "numfiles" INTEGER,\n  "numdir" INTEGER,\n  "tamano" INTEGER,\n  "libre" INTEGER,\n  "fechaudit" TEXT\n)')
('table', 'articulos', 'articulos', 1972, 'CREATE TABLE articulos (\n                              codigo integer primary key autoincrement,\n                              de

### IMPORTANTE - Cerrar conexión con BD

In [ ]:
# No te olvides de cerrar la conexión
bd.close()